In [1]:
!pip install -U -q --no-cache-dir ultralytics ipywidgets torchinfo albumentations

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
import shutil

original_folder = '/opt/conda/lib/python3.10/site-packages/ultralytics'
custom_folder = '/kaggle/input/rtdetrv2-ultralytics/ultralytics'

shutil.rmtree(original_folder)

shutil.copytree(custom_folder, original_folder)

'/opt/conda/lib/python3.10/site-packages/ultralytics'

In [3]:
import yaml
import cv2
import numpy as np
import json
import glob
import random
import shutil
import gc
import matplotlib.pyplot as plt
import os
import torch
from tqdm import tqdm
from ultralytics import YOLO, RTDETR

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from torchinfo import summary

In [5]:
torch.cuda.empty_cache()
gc.collect()

0

In [6]:
# data_yaml = {
#     'train': '/kaggle/input/vehicle-detection-v1/combination/combination/images/train',
#     'val': '/kaggle/input/vehicle-detection-v1/combination/combination/images/val',
#     'names': ["bus", "car", "motorbike", "truck"],
#     'augmentation': {
#         'hsv_h': 0.015,
#         'hsv_s': 0.7, 
#         'hsv_v': 0.4,
#         'flipud': 0.1, 
#         'fliplr': 0.5,
#         'mosaic': 1.0,
#         'mixup': 0.0,
#         'scale': 0.5,
#         'translate': 0.1,
#         'perspective': 0.0,
#         'blur': 0.2
#     }
# }

# with open('/kaggle/working/data.yaml', 'w') as file:
#     yaml.dump(data_yaml, file, default_flow_style=False)

In [7]:
data_yaml = {
    'train': '/kaggle/input/vn-license-detection/VN license detection/images/train',
    'val': '/kaggle/input/vn-license-detection/VN license detection/images/val',
    'names': ["license_plate"]
}

with open('/kaggle/working/data.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [8]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 72.8MB/s]


In [9]:
model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [10]:
# model.train(data='/kaggle/working/data.yaml', epochs=100, workers=4, batch=128, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1], amp=True, warmup_epochs=10, deterministic=False, lr0=0.01, lrf=0.0001)

In [11]:
model.train(data='/kaggle/working/data.yaml', epochs=100, augment=False,workers=4, batch=64, imgsz=640, seed=42, optimizer="AdamW", save_period=10, device=[0, 1], amp=False, warmup_epochs=10, lr0=0.01, lrf=0.001)

New https://pypi.org/project/ultralytics/8.3.145 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=10, cache=False, device=[0, 1], workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/vn-license-detection/VN license detection/labels/train... 10039 images, 0 backgrounds, 1 corrupt: 100%|██████████| 10040/10040 [00:34<00:00, 295.17it/s]


train: WARNING ⚠️ /kaggle/input/vn-license-detection/VN license detection/images/train/quandoi63.jpg: ignoring corrupt image/label: [Errno 30] Read-only file system: '/kaggle/input/vn-license-detection/VN license detection/images/train/quandoi63.jpg'
train: WARNING ⚠️ Cache directory /kaggle/input/vn-license-detection/VN license detection/labels is not writeable, cache not saved.
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 3887, len(boxes) = 10670. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all       2797       2980      0.992      0.989      0.994      0.835

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      7.77G      0.741     0.2724     0.9383         28        640: 100%|██████████| 157/157 [01:45<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:19<00:00,  2.25it/s]


                   all       2797       2980      0.993      0.989      0.994      0.834

100 epochs completed in 3.509 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:20<00:00,  2.12it/s]


                   all       2797       2980      0.991      0.989      0.994      0.835
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train
